### **Задание №1**. Предположим, что у вас в базе есть табличка с балансом по всем счетам клиента на конец дня. Из-за особенностей построения таблицы, часть значений незаполнена и физически это означает отсутствие информации об изменении баланса. Для простоты будем считать, пропущенные значения для одного пользователя не могут встретиться между двумя заполненными. Напишите SQL запрос для заполнения неизвестных значений по каждому клиенту последним известным.

client_id |date | value
--- | --- | ---
1 | 14.04.2020 | 10000
1 | 15.04.2020 | 5000
1 | 16.04.2020 | NaN
1 | 17.04.2020 | NaN
1 | 18.04.2020 | NaN
2 | 14.04.2020 | 250000
2 | 15.04.2020 | 250000
2 | 16.04.2020 | 230000
2 | 17.04.2020 | 230000
2 | 18.04.2020 | 225000
3 | 14.04.2020 | 50000
3 | 15.04.2020 | NaN
3 | 16.04.2020 | NaN
3 | 17.04.2020 | NaN
3 | 18.04.2020 | NaN


In [ ]:
# <YOUR_CODE_HERE>
SELECT client_id,date,
COALESCE(Value,
 		(SELECT value  FROM client t2
						WHERE t1.client_id=t2.client_id  AND t2.date<t1.date
									AND t2.value IS NOT NULL ORDER BY t2.DATE DESC LIMIT 1)) AS value
FROM client t1

#решение 1.1 не отличаетяся от решения 1. Использовал PostgreSQL, в нем нельзя исползовать ignore_null для оконной фнукции lag 

### **Задание №1.1 (*)**. В прошлом задании мы считали, что пропущенные значения не могут находиться между двумя заполненными в рамках одного пользователя, что не всегда правда в реальных задачах. Напишите теперь SQL запрос без указанного допущения.

client_id |date | value
--- | --- | ---
1 | 14.04.2020 | 10000
1 | 15.04.2020 | 5000
1 | 16.04.2020 | NaN
1 | 17.04.2020 | 7000
1 | 18.04.2020 | NaN
2 | 14.04.2020 | 250000
2 | 15.04.2020 | 250000
2 | 16.04.2020 | 230000
2 | 17.04.2020 | 230000
2 | 18.04.2020 | 225000
3 | 14.04.2020 | 50000
3 | 15.04.2020 | NaN
3 | 16.04.2020 | NaN
3 | 17.04.2020 | 49000
3 | 18.04.2020 | NaN

In [ ]:
# <YOUR_CODE_HERE>

SELECT client_id,date,
COALESCE (t1.value,(SELECT value FROM client_1 t2 
			WHERE t2.client_id=t1.client_id AND t2.date<t1.date AND t2.value 
			IS NOT NULL ORDER BY t2.date  DESC LIMIT 1)) real_value
FROM client_1  t1

#решение 1.1 не отличаетяся от решения 1. Использовал PostgreSQL, в нем нельзя исползовать ignore_null для оконной фнукции lag 


### **Задание №2**. Пусть клиенты обладают рядом статусов, закодированных цифрами: 0 - "улица", 1 - "зарплатник", 2 - "премиум", 3 - "пенсионер", 4 - "студент". У нас есть матрица с логами смен статусов за некоторый период, разверните ее в матрицу статус - статус с частотами перехода между ними.

Пример, как было:

client_id | prev_segment | new_segment|
--- | --- | ---
1 | 0 | 1
2 | 0 | 2
3 | 4 | 3

Пример, как нужно:

segment |0 | 1 | 2 | 3 | 4 
--- | --- | --- | --- | --- | ---
0 | 0 | 0.5 | 0.5 | 0 | 0 
1 | 0 |0 | 0 | 0 | 0
2 | 0 | 0 | 0 | 0 | 0
3 | 0 | 0 | 0 | 0 | 0
4 | 0 | 0 | 0 | 1 | 0

В процессе выполнения задания разрешается пользоваться библиотеками numpy и pandas.

In [ ]:
import numpy as np
import pandas as pd

# Сгенерируем данные для задачи
data = pd.DataFrame(
    np.random.randint(low=0, high=5, size=(50000000, 2)),
    columns=['prev_segment', 'new_segment']
)

# <YOUR_CODE_HERE>

data=data.reset_index().rename(columns={'index':'client_id'})

# Create a pivot table with counts of transitions between statuses
pivot_table = pd.pivot_table(data, values='client_id', index='prev_segment', columns='new_segment', aggfunc='count', fill_value=0)


pivot_table['sum']=pivot_table.sum(axis=1)

for col in pivot_table.columns:
    pivot_table[col]=(pivot_table[col]/pivot_table['sum']).round(3)
pivot_table=pivot_table.drop(columns='sum')

display(pivot_table)

new_segment,0,1,2,3,4
prev_segment,,,,,
0,0.2,0.2,0.2,0.2,0.2
1,0.2,0.2,0.2,0.2,0.2
2,0.2,0.2,0.2,0.2,0.2
3,0.2,0.2,0.2,0.2,0.2
4,0.2,0.2,0.2,0.2,0.2


### **Задание №2.1 (*)**. Если вы не пользовались библиотечными функциями и реализовывали алгоритм самостоятельно, напишите, пожалуйста, его асимптотическую сложность в терминах O-большое.

In [ ]:
# <YOUR_CODE_HERE>

### **Задание №3**. Постройте прогнозую модель для датасета с UCI Machine Learning: предобработайте категориальные фичи, отберите их, соберите схему валидации и отберите модели с помощью нее. Описание датасета по ссылке: https://archive.ics.uci.edu/ml/datasets/Drug+consumption+%28quantified%29

Данная задача multilabel multiclass classification, поэтому предлагаю начать с одного из подходов:
- Свести задачу к multilabel binary classification, как это указано в описании к датасету;
- Свести задачку к binary classification тем или иным способом;
- Обучить множество multiclass/binary classification моделей;
- Что-то еще, что придет в голову.


In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00373/drug_consumption.data

--2021-04-20 07:39:48--  https://archive.ics.uci.edu/ml/machine-learning-databases/00373/drug_consumption.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346695 (339K) [application/x-httpd-php]
Saving to: ‘drug_consumption.data’

drug_consumption.da 100%[===================>] 338.57K  1.28MB/s    in 0.3s    

2021-04-20 07:39:48 (1.28 MB/s) - ‘drug_consumption.data’ saved [346695/346695]



In [ ]:
import numpy as np
import pandas as pd

data = pd.read_csv('drug_consumption.data', header=None, index_col=0)
data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,CL5,CL2,CL0,CL2,CL6,CL0,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,CL5,CL2,CL2,CL0,CL6,CL4,CL6,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,CL6,CL0,CL0,CL0,CL6,CL3,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,CL4,CL0,CL0,CL3,CL5,CL2,CL4,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,CL4,CL1,CL1,CL0,CL6,CL3,CL6,CL0,CL0,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0


In [ ]:
# <YOUR_CODE_HERE> Выложил отдельным ноутбуком по ссылке ниже, чтобы было удобней читать
#https://colab.research.google.com/drive/1qA0pBHeiEojW24B735adR0Fel6mKmhfV?usp=sharing

### **Задание №4**. Предположим, что мы построили рекомендательную систему, и ожидаем, что средний чек клиента вследствие этого должен вырасти. Мы провели AB-тест и собрали данные по контрольной и тестовой группе. Предложите и проведите статистический тест, который можно применить к этим данным.

В процессе выполнения задания разрешено пользоваться библиотеками scipy и statsmodels.

In [ ]:
import numpy as np

# Сгенерируем данные для задачи
control = np.random.lognormal(mean=5, sigma=2, size=(100000,))
test = np.random.lognormal(mean=5.04, sigma=2, size=(100000,))

In [ ]:
# <YOUR_CODE_HERE>
from scipy.stats import mannwhitneyu
from tqdm import tqdm

n=1000
alpha = 0.05
beta = 0.2
p_vals =[]
for _ in tqdm(range(n)):
    control = np.random.lognormal(mean=5, sigma=2, size=(100000,))
    test = np.random.lognormal(mean=5.04, sigma=2, size=(100000,))
    p_vals.append(mannwhitneyu(control,test,alternative='two-sided')[1])


print('Статистическая мощность: ',(np.array(p_vals)<0.05).mean())


if (np.array(p_vals)<alpha).mean() >(1 - beta):
    print("Отвергаем нулевую гипотезу, различия между группами статистически значимы")
else:
    print("Не удалось отвергнуть нулевую гипотезу, различия между группами не являются статистически значимыми")

100%|██████████| 1000/1000 [04:15<00:00,  3.91it/s]

Статистическая мощность:  0.991
Отвергаем нулевую гипотезу, различия между группами статистически значимы


### **Задание №4 (*)**. В прошлом задании мы улавливали статистически значимое различие в средних в 0.04 единицы на выборке из 100000 клиентов. Насколько больше нам требуется собрать клиентов, чтобы уловить различие в 0.02?

In [ ]:
# <YOUR_CODE_HERE>

n=1000
alpha = 0.05
beta = 0.2
#sample_sizes = range(100000,200000,5000) слишком долго считается
sample_sizes = range(160000,170000,5000)
for sample_size in sample_sizes:
    p_vals =[]
    for _ in tqdm(range(n)):
        control = np.random.lognormal(mean=5, sigma=2, size=(sample_size,))
        test = np.random.lognormal(mean=5.02, sigma=2, size=(sample_size,))
        p_vals.append(mannwhitneyu(control,test,alternative='two-sided')[1])

    print('Размер сэмпла',sample_size)
    print('Статистическая мощность: ',(np.array(p_vals)<0.05).mean())



#Математическую формулу к сожалению не нашел. Перебором получилось что выборка нужна в районе 165 000 наблюдений

100%|██████████| 1000/1000 [07:01<00:00,  2.37it/s]


Размер сэмпла 160000
Статистическая мощность:  0.785


100%|██████████| 1000/1000 [07:14<00:00,  2.30it/s]

Размер сэмпла 165000
Статистическая мощность:  0.823
